In [1]:
import tensorflow as tf

model_dir = 'new_app/finetuned_ed0/saved_model/'  # Ensure to end with '/'
loaded_model = tf.saved_model.load(model_dir)

2023-08-18 15:08:04.844699: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-08-18 15:08:04.875774: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-18 15:08:05.441430: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-08-18 15:08:06.449099: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.

In [3]:
def load_labelmap(labelmap_path):
    category_map = {}
    with open(labelmap_path, 'r') as file:
        lines = file.readlines()
        current_id = None
        current_name = None
        for line in lines:
            line = line.strip()
            if "id:" in line:
                current_id = int(line.split(":")[1].strip())
            if "name:" in line:
                current_name = line.split(":")[1].strip().replace("'", "")
                category_map[current_id] = {'id': current_id, 'name': current_name}
    return category_map

category_map = load_labelmap("new_app/train/labelmap.txt")
print(category_map)

{1: {'id': 1, 'name': 'kalb'}, 2: {'id': 2, 'name': 'person'}}


In [4]:
def run_inference_for_single_image(model, image):
    image = np.asarray(image)
    # Convert the image to a tensor.
    input_tensor = tf.convert_to_tensor(image)
    # The model expects a batch of images, so add an axis with `tf.newaxis`.
    input_tensor = input_tensor[tf.newaxis,...]

    # Run inference
    detections = model(input_tensor)

    return detections


In [5]:
import numpy as np
from PIL import Image
from object_detection.utils import visualization_utils as viz_utils

image_path = "new_app/train/2019-08-03_090001.jpg"
image_np = np.array(Image.open(image_path))

# Actual detection.
output_dict = run_inference_for_single_image(loaded_model, image_np)
print(output_dict['detection_scores'][0].numpy())

# Visualization of the results of a detection.
viz_utils.visualize_boxes_and_labels_on_image_array(
    image_np,
    output_dict['detection_boxes'][0].numpy(),
    output_dict['detection_classes'][0].numpy().astype(np.int32),
    output_dict['detection_scores'][0].numpy(),
    category_map,
    use_normalized_coordinates=True,
    line_thickness=8,
    min_score_thresh=0.045 )

# # Display
Image.fromarray(image_np).show()


2023-08-18 15:08:36.809016: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8600
2023-08-18 15:08:36.862645: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-08-18 15:08:36.863395: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-08-18 15:08:36.863416: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:109] Couldn't get ptxas version : FAILED_PRECONDITION: Couldn't get ptxas/nvlink version string: INTERNAL: Couldn't invoke ptxas --version
2023-08-18 15:08:36.864168: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-08-18 15:08:36.864198: W tensorflow/compiler/xla/stream_executor/gpu/redzone_allocator.cc:318] INTERNAL: Failed to launch ptxas
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This mes

[0.25675145 0.25008643 0.17582943 0.15982264 0.12070093 0.12034408
 0.10581971 0.09777072 0.09723876 0.09175884 0.09089711 0.08759634
 0.08483321 0.08388683 0.08369277 0.08261616 0.08204383 0.08051664
 0.07735747 0.06836545 0.06721197 0.06402599 0.06395116 0.06329389
 0.06183547 0.05948424 0.05555177 0.05512119 0.05488646 0.05230032
 0.05223281 0.05203326 0.05163156 0.0510985  0.05040763 0.05007048
 0.04910192 0.04852357 0.04833131 0.0482905  0.04646717 0.04612269
 0.04493431 0.04437536 0.04432748 0.04418956 0.04401806 0.04400184
 0.04397394 0.04383481 0.04371039 0.04354383 0.04271274 0.0425141
 0.0420298  0.04194007 0.04120698 0.04105372 0.04012007 0.04003285
 0.03992224 0.03949263 0.03934186 0.03923856 0.03903082 0.03894954
 0.0381141  0.03807874 0.03795916 0.03733015 0.03679034 0.03654511
 0.03647829 0.03570371 0.03544566 0.03537332 0.03530805 0.0352655
 0.03521949 0.03500531 0.03492214 0.03465002 0.03421936 0.03418078
 0.03417576 0.03406192 0.03402057 0.03342976 0.03329136 0.033274